# 0.0 LangGraph 快速案例：飞速上手指南

## 前言：如何才能快速上手？

案例运行和调试，是最佳的编程入门方式，也是本书写作的核心理念。通过案例运行，我们可以快速见识到 LangGraph 和 Python 相关的概念和知识点，然后我们再逐步剖析，反复推敲，甚至一日三复习，直至熟练掌握。

那什么又叫"熟练掌握"呢，就是能用通俗语言，向外行人或者新手解释，条理清晰娓娓道来；或像在面试官面前侃侃而谈，从从容容游刃有余。

因为我们要想到一个道理，对于一门新兴的技术，只要你愿意花时间去见识和打磨，你就大概率懂得比别人多，比同行多，甚至比负责技术的面试官更多。

在 AI Agent 开发的世界里，**理论千篇一律，通俗的解读万里挑一**。本章的设计理念是：**让你在 5 分钟内跑起第一个 Agent，在 30 分钟内理解核心模式，在 1 小时内见识"多 Agent 协作"**。

我们精心设计了 5 个从简单到复杂的案例，每个案例都：
- ✅ **代码精简**：核心代码不超过 100 行
- ✅ **开箱即用**：复制粘贴即可运行
- ✅ **循序渐进**：每个案例只引入 1-2 个新概念
- ✅ **实战导向**：所有案例都基于官方案例，或者真实的业务场景

**学习路径**：
```
案例 1: 基础聊天机器人 (5 分钟)
  ↓ 引入状态管理
案例 2: 带记忆的聊天机器人 (10 分钟)
  ↓ 引入工具调用
案例 3: 工具调用 Agent (15 分钟)
  ↓ 引入 ReAct 推理
案例 4: ReAct 搜索 Agent (20 分钟)
  ↓ 引入多 Agent 协作
案例 5: 多 Agent 协作系统 (30 分钟)
```

## 环境准备

### 安装依赖

In [5]:
# 核心依赖
!pip install -U pip setuptools wheel -q 

In [ ]:
!pip install -U langchain langchain-core langchain-openai langgraph transformers importlib-metadata -q

In [ ]:
# 在 Notebook 中使用 nest_asyncio（推荐）。这个库可以让你“嵌套”异步循环，让 Pyppeteer 能正常运行
pip install nest_asyncio 

In [ ]:
pip install pyppeteer

In [1]:
## 检查安装的核心包及其版本
import importlib

for pkg in ["langgraph", "langchain", "openai", "transformers"]:
    try:
        version = importlib.import_module(pkg).__version__
        print(f"{pkg}: {version}")
    except ImportError:
        print(f"{pkg}: 未安装")
    except AttributeError:
        print(f"{pkg}: 已安装，但未提供版本号")

langgraph: 已安装，但未提供版本号
langchain: 0.3.27
openai: 2.3.0


/Users/brycewang/langgraph-lightning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


transformers: 4.57.0


### API 密钥配置

本章所有案例都使用 `getpass()` 来安全地读取 API 密钥，避免硬编码泄露：

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 在每个案例开头调用
_set_env("OPENAI_API_KEY")

### 去掉运行时的警告

In [3]:
import warnings
warnings.filterwarnings("ignore")

## 案例 1: 最简单的聊天机器人

**目标**：用 10 行代码构建一个能对话的聊天机器人  
**核心概念**：`StateGraph`、`MessagesState`、节点函数

### 完整代码

In [11]:
# 导入标准库，用于安全输入环境变量（API key）
import getpass
import os

# 定义一个函数，用于设置环境变量（比如 OPENAI_API_KEY）
def _set_env(var: str):
    # 如果系统环境中还没有这个变量
    if not os.environ.get(var):
        # 通过命令行安全输入（不会在屏幕显示）
        os.environ[var] = getpass.getpass(f"{var}: ")

# 调用函数，确保设置 OpenAI 的 API Key
_set_env("OPENAI_API_KEY")

# 导入 LangChain 的 OpenAI 模型封装类
from langchain_openai import ChatOpenAI

# 从 LangGraph 导入关键组件
# StateGraph：图结构构建器
# MessagesState：图中状态的结构定义（专门处理“消息对话”类状态）
# START / END：图的起点与终点标识
from langgraph.graph import StateGraph, MessagesState, START, END

# -------------------------------
# 定义一个“节点”函数 —— chatbot
# 每个节点函数都接收并返回一个状态（这里是 MessagesState）
# -------------------------------
def chatbot(state: MessagesState):
    # 使用 ChatOpenAI 调用一个轻量模型（gpt-5-nano）
    # 将输入状态中的“消息”作为模型输入
    reply = ChatOpenAI(model="gpt-5-nano").invoke(state["messages"])
    # 返回一个新的状态字典（LangGraph 允许增量式更新）
    return {"messages": [reply]}

# -------------------------------
# 构建图（Graph）结构
# -------------------------------
# 创建一个基于消息状态的图
graph = StateGraph(MessagesState)

# 向图中添加一个节点，命名为 "chatbot"，并绑定对应函数
graph.add_node("chatbot", chatbot)

# 添加边，定义节点执行顺序
# 图结构为：START → chatbot → END
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# -------------------------------
# 编译并运行
# -------------------------------
# 把图结构“编译”成一个可执行的应用对象
app = graph.compile()

# 调用应用：输入一条用户消息
response = app.invoke({"messages": [("user", "你好，请用一句话介绍 LangGraph")]})

# 输出模型回复内容
print(response["messages"][-1].content)

LangGraph 是一个用于自然语言处理和图数据处理的框架，能够有效地将语言信息与图结构结合，支持多种文本分析和图挖掘任务。


In [ ]:
## 经典的打印 LangGraph 图架构的方式
from IPython.display import Image, display
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

#display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
## 查看简单版的 graph 架构
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	chatbot --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



### 运行结果

```
LangGraph 是一个用于构建和管理复杂语言模型工作流的框架，支持状态管理和多步骤推理。
```

### 关键点解析

1. **StateGraph**：图的容器，定义工作流结构
2. **MessagesState**：内置状态类型，自动管理对话历史
3. **节点函数**：接收 `state`，返回新的状态更新
4. **START/END**：特殊节点，标记图的入口和出口

¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥¥

## 案例 2: 带记忆的聊天机器人

**目标**：让 Agent 记住对话历史，支持多轮对话  
**核心概念**：`MemorySaver`、检查点持久化

### 完整代码

In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

# 定义节点
def chatbot(state: MessagesState):
    return {"messages": [ChatOpenAI(model="gpt-5-nano").invoke(state["messages"])]}

# 构建图（添加记忆）
graph = StateGraph(MessagesState)
graph.add_node("chatbot", chatbot)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# 使用 MemorySaver 保存对话历史
memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# 多轮对话
config = {"configurable": {"thread_id": "user_001"}}

# 第一轮
response1 = app.invoke(
    {"messages": [("user", "我的名字是小明")]},
    config=config
)
print("Round 1:", response1["messages"][-1].content)

# 第二轮（Agent 应该记住名字）
response2 = app.invoke(
    {"messages": [("user", "我的名字是什么？")]},
    config=config
)
print("Round 2:", response2["messages"][-1].content)

### 运行结果

```
Round 1: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？
Round 2: 你的名字是小明。
```

### 关键点解析

1. **MemorySaver**：内存检查点保存器，存储对话历史
2. **thread_id**：会话标识符，不同用户使用不同 ID
3. **自动累积**：`MessagesState` 会自动将新消息追加到历史中

## 案例 3: 工具调用 Agent

**目标**：让 Agent 能够调用外部工具（计算器、天气查询）  
**核心概念**：工具绑定、条件边、工具节点

### 完整代码

In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

# 定义工具
@tool
def calculator(expression: str) -> str:
    """执行数学计算。输入：数学表达式（如 '2+3*4'）"""
    try:
        return str(eval(expression))
    except:
        return "计算错误"

@tool
def get_weather(city: str) -> str:
    """查询城市天气。输入：城市名称"""
    # 模拟天气查询
    weather_db = {
        "北京": "晴天，15°C",
        "上海": "多云，18°C",
        "深圳": "阴天，22°C"
    }
    return weather_db.get(city, "未知城市")

tools = [calculator, get_weather]

# 定义节点
llm = ChatOpenAI(model="gpt-5-nano")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 条件边：判断是否需要调用工具
def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# 构建图
graph = StateGraph(MessagesState)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot", should_continue, ["tools", END])
graph.add_edge("tools", "chatbot")

app = graph.compile()

# 测试
response = app.invoke({
    "messages": [("user", "北京的天气怎么样？计算一下 25 * 4 + 10")]
})
print(response["messages"][-1].content)

### 运行结果

```
北京今天是晴天，气温 15°C。计算结果：25 * 4 + 10 = 110。
```

### 关键点解析

1. **@tool 装饰器**：将 Python 函数转换为 LLM 可调用的工具
2. **bind_tools()**：将工具绑定到 LLM，使其知道可以调用哪些工具
3. **ToolNode**：专门执行工具调用的节点
4. **条件边**：根据 LLM 的输出（是否有 tool_calls）动态路由
5. **循环**：tools → chatbot 形成循环，支持多次工具调用

## 案例 4: ReAct 推理 Agent

**目标**：构建能"思考-行动-观察"的 Agent，集成真实搜索引擎  
**核心概念**：ReAct 模式、Tavily 搜索、系统提示词

### 完整代码

In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")  # 注册：https://tavily.com

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

# 初始化工具
search = TavilySearchResults(max_results=2)
tools = [search]

# 定义 Agent（添加 ReAct 系统提示词）
llm = ChatOpenAI(model="gpt-5-nano")
llm_with_tools = llm.bind_tools(tools)

def agent(state: MessagesState):
    # 添加 ReAct 提示词
    system_message = """你是一个 ReAct (Reasoning + Acting) Agent。
处理用户问题时，请遵循以下步骤：
1. Thought（思考）：分析问题需要什么信息
2. Action（行动）：决定调用哪个工具
3. Observation（观察）：分析工具返回的结果
4. Answer（回答）：基于观察给出最终答案

始终展示你的推理过程。"""

    messages = [{"role": "system", "content": system_message}] + state["messages"]
    return {"messages": [llm_with_tools.invoke(messages)]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# 构建图
graph = StateGraph(MessagesState)
graph.add_node("agent", agent)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "agent")
graph.add_conditional_edges("agent", should_continue, ["tools", END])
graph.add_edge("tools", "agent")

app = graph.compile()

# 测试
response = app.invoke({
    "messages": [("user", "2024年诺贝尔物理学奖获得者是谁？他们的主要贡献是什么？")]
})
print(response["messages"][-1].content)

### 运行结果

```
**Thought**: 用户询问 2024 年诺贝尔物理学奖的获得者，我需要搜索最新信息。

**Action**: 使用搜索工具查询 "2024 Nobel Prize Physics winner"。

**Observation**: 搜索结果显示，2024 年诺贝尔物理学奖授予了 John Hopfield 和 Geoffrey Hinton，
表彰他们在人工神经网络领域的开创性发现。

**Answer**: 2024 年诺贝尔物理学奖的获得者是 John Hopfield 和 Geoffrey Hinton。
他们的主要贡献是：
- Hopfield: 发明了 Hopfield 网络，一种可以存储和重建模式的联想记忆网络
- Hinton: 开发了反向传播算法和深度学习的关键技术，被誉为"深度学习之父"
```

### 关键点解析

1. **ReAct 模式**：通过系统提示词引导 LLM 展示推理过程
2. **Tavily 搜索**：企业级搜索 API，专为 AI Agent 优化
3. **循环推理**：Agent 可以多次调用工具，逐步完善答案

## 案例 5: 多 Agent 协作系统

**目标**：构建"研究员 + 图表分析师 + 报告撰写员"的团队协作系统  
**核心概念**：多 Agent、消息路由、子图

### 完整代码

In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from typing import Literal

# 定义 3 个专业 Agent
llm = ChatOpenAI(model="gpt-5-nano")

def researcher(state: MessagesState):
    """研究员：负责信息收集"""
    system_msg = "你是资深研究员，擅长收集和分析行业信息。请提供数据和趋势分析。"
    messages = [{"role": "system", "content": system_msg}] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

def chart_analyst(state: MessagesState):
    """图表分析师：负责数据可视化建议"""
    system_msg = "你是数据可视化专家，擅长将数据转化为图表建议。请推荐合适的图表类型和关键指标。"
    messages = [{"role": "system", "content": system_msg}] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

def report_writer(state: MessagesState):
    """报告撰写员：整合信息并生成最终报告"""
    system_msg = "你是专业报告撰写员，擅长将研究结果和图表建议整合成结构清晰的报告。"
    messages = [{"role": "system", "content": system_msg}] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

# Supervisor：决定下一步调用哪个 Agent
def supervisor(state: MessagesState) -> Literal["researcher", "chart_analyst", "report_writer", "end"]:
    """管理者：协调各个 Agent 的工作流程"""
    messages = state["messages"]

    # 简单的状态机逻辑
    user_message = messages[0].content if messages else ""
    response_count = len([m for m in messages if hasattr(m, "response_metadata")])

    if response_count == 0:
        return "researcher"  # 第一步：研究
    elif response_count == 1:
        return "chart_analyst"  # 第二步：图表分析
    elif response_count == 2:
        return "report_writer"  # 第三步：报告撰写
    else:
        return "end"  # 完成

# 构建图
graph = StateGraph(MessagesState)

# 添加节点
graph.add_node("researcher", researcher)
graph.add_node("chart_analyst", chart_analyst)
graph.add_node("report_writer", report_writer)

# 添加边（串行工作流）
graph.add_edge(START, "researcher")
graph.add_edge("researcher", "chart_analyst")
graph.add_edge("chart_analyst", "report_writer")
graph.add_edge("report_writer", END)

app = graph.compile()

# 测试
response = app.invoke({
    "messages": [("user", "请帮我分析一下 2024 年生成式 AI 市场的发展趋势，并给出报告")]
})

# 打印每个 Agent 的输出
print("=== 研究员输出 ===")
print(response["messages"][1].content[:200] + "...\n")

print("=== 图表分析师输出 ===")
print(response["messages"][2].content[:200] + "...\n")

print("=== 最终报告 ===")
print(response["messages"][3].content)

### 运行结果

```
=== 研究员输出 ===
根据市场研究，2024 年生成式 AI 市场呈现以下趋势：
1. 市场规模预计达到 500 亿美元，同比增长 150%
2. 主要应用领域：内容创作（40%）、代码生成（25%）、客户服务（20%）
3. 技术突破：多模态模型、实时推理、成本优化...

=== 图表分析师输出 ===
针对上述数据，建议使用以下可视化方案：
1. **市场规模趋势图**（折线图）：展示 2022-2024 年市场增长曲线
2. **应用领域占比**（饼图）：突出内容创作的主导地位
3. **技术成熟度雷达图**：对比多模态、实时推理等技术的成熟度...

=== 最终报告 ===
# 2024 年生成式 AI 市场发展趋势报告

## 一、市场概况
2024 年生成式 AI 市场规模预计达到 500 亿美元，同比增长 150%，标志着该行业进入快速发展期。

## 二、应用分布
- **内容创作**（40%）：文本、图像、视频生成
- **代码生成**（25%）：GitHub Copilot、Cursor 等工具
- **客户服务**（20%）：智能客服、对话机器人

## 三、数据可视化建议
建议使用折线图展示市场增长曲线，饼图突出应用领域占比，雷达图对比技术成熟度。

## 四、结论与展望
生成式 AI 正在从技术创新走向商业化落地，预计 2025 年将出现更多杀手级应用。
```

### 关键点解析

1. **专业化分工**：每个 Agent 有明确的角色和系统提示词
2. **串行工作流**：researcher → chart_analyst → report_writer
3. **状态累积**：每个 Agent 都能看到前面所有的对话历史
4. **可扩展性**：可以轻松添加更多专业 Agent（如 data_validator、editor）

### 进阶扩展

更复杂的多 Agent 系统可以使用：
- **并行执行**：多个 Agent 同时工作，提高效率
- **动态路由**：Supervisor 根据任务复杂度动态选择 Agent
- **人机协作**：在关键节点加入人工审核断点

## 官方案例库精选解读

LangGraph 官方仓库（https://github.com/langchain-ai/langgraph/tree/main/examples）包含 50+ 个生产级案例，涵盖各种应用场景。以下是分类精选：

### 🤖 Agent 类型

#### 1. **chatbot-simulation-evaluation/**
- **功能**：评估聊天机器人质量的模拟框架
- **适用场景**：产品上线前的自动化测试
- **核心技术**：模拟用户对话、多维度评分

#### 2. **multi-agent/**
- **功能**：多 Agent 协作的完整实现
- **适用场景**：复杂任务分解、团队协作模拟
- **核心技术**：Supervisor 模式、消息路由

#### 3. **customer-support/**
- **功能**：企业级客服系统
- **适用场景**：电商、SaaS 客户支持
- **核心技术**：意图识别、工单系统集成、情感分析

---

### 🧠 高级技术

#### 4. **rag/**
- **功能**：检索增强生成（Retrieval-Augmented Generation）
- **适用场景**：知识库问答、文档助手
- **核心技术**：向量检索、上下文注入、引用溯源
- **推荐指数**：⭐⭐⭐⭐⭐（最常用模式）

#### 5. **reflection/**
- **功能**：Agent 自我反思和优化
- **适用场景**：代码审查、内容校对、论文撰写
- **核心技术**：自我批评、迭代改进

#### 6. **plan-execute/**
- **功能**：先规划再执行的任务分解模式
- **适用场景**：复杂项目管理、多步骤任务
- **核心技术**：任务拆解、依赖管理、动态调整

#### 7. **rewoo/**
- **功能**：推理而不观察（Reasoning Without Observation）
- **适用场景**：需要提前规划的任务（如旅行规划）
- **核心技术**：预先生成完整计划、批量工具调用

---

### 🔧 技术能力

#### 8. **memory/**
- **功能**：多种记忆机制实现
- **适用场景**：长期对话、用户画像构建
- **核心技术**：短期记忆、长期记忆、记忆检索

#### 9. **tool-calling/**
- **功能**：工具调用的各种模式
- **适用场景**：API 集成、外部服务调用
- **核心技术**：并行工具调用、错误重试、结果验证

#### 10. **streaming/**
- **功能**：流式输出和中间结果展示
- **适用场景**：实时对话、长文本生成
- **核心技术**：Token 流式传输、进度反馈

#### 11. **persistence/**
- **功能**：状态持久化和恢复
- **适用场景**：长时间运行任务、断点续传
- **核心技术**：CheckPoint、状态序列化

---

### 🌐 专业领域

#### 12. **web-navigation/**
- **功能**：自动化网页浏览和信息抽取
- **适用场景**：网页爬虫、竞品分析、价格监控
- **核心技术**：Playwright 集成、DOM 解析

#### 13. **extraction/**
- **功能**：结构化信息抽取
- **适用场景**：简历解析、合同要素提取、发票识别
- **核心技术**：Schema 定义、字段验证、批量处理

#### 14. **code-assistant/**
- **功能**：代码生成和重构助手
- **适用场景**：IDE 插件、代码审查、自动补全
- **核心技术**：AST 解析、语法检查、单元测试生成

---

### 📊 评估与监控

#### 15. **llm-compiler/**
- **功能**：并行优化的 Agent 执行引擎
- **适用场景**：高性能要求的生产环境
- **核心技术**：任务 DAG、并行调度、依赖管理

#### 16. **chatbot-simulation-evaluation/**
- **功能**：自动化评估框架
- **适用场景**：AB 测试、模型选型、提示词优化
- **核心技术**：模拟对话、指标计算、统计分析

---

### 🎯 推荐学习路径

如果你是新手，建议按以下顺序学习官方案例：

```
第 1 层：基础模式
  ├─ chatbot/                    # 基础对话
  ├─ tool-calling/                # 工具集成
  └─ streaming/                   # 流式输出

第 2 层：高级技术
  ├─ rag/                         # 检索增强（必学）
  ├─ memory/                      # 记忆管理
  └─ reflection/                  # 自我优化

第 3 层：复杂系统
  ├─ multi-agent/                 # 多 Agent 协作
  ├─ plan-execute/                # 任务规划
  └─ customer-support/            # 完整业务案例

第 4 层：专业领域
  ├─ web-navigation/              # 网页自动化
  ├─ extraction/                  # 信息抽取
  └─ code-assistant/              # 代码助手
```

---

### 💡 官方案例的学习建议

1. **克隆仓库**：
   ```bash
   git clone https://github.com/langchain-ai/langgraph.git
   cd langgraph/examples
   ```

2. **按需运行**：
   - 每个案例都有独立的 README 和依赖说明
   - 建议先阅读 README，理解设计思路
   - 再运行代码，观察实际效果

3. **改造实践**：
   - 不要只是运行官方代码
   - 尝试修改提示词、添加新工具、调整工作流
   - 将案例迁移到你的实际业务场景

4. **源码阅读**：
   - 官方案例代码质量极高，包含大量最佳实践
   - 重点关注错误处理、日志记录、性能优化
   - 学习代码结构和模块划分

## 总结与下一步

### 你已经见识到的核心技能

通过本章的 5 个案例，你已经初步见识到：

✅ **案例 1**：StateGraph 基础、消息流转
✅ **案例 2**：检查点持久化、多轮对话
✅ **案例 3**：工具调用、条件路由
✅ **案例 4**：ReAct 推理、搜索集成
✅ **案例 5**：多 Agent 协作、角色分工

### 从"飞速上手"到"真正掌握"

本章的设计是"快速起步"，让大大直观看看并上手 LangGraph。显然，LangGraph 的能力远不止于此， LangGraph 的核心概念我们还将反复讲解，帮助大家完全消化。

接下来的章节将深入讲解：

- **第 1 章**：Chain、Router、Agent 的本质区别
- **第 2 章**：State Schema 的高级设计模式
- **第 3 章**：断点调试和时间旅行
- **第 4 章**：并行执行、子图、Map-Reduce
- **第 5-6 章**：生产部署、监控、安全性

### 实战练习建议

在继续学习之前，强烈建议完成以下练习：

1. **改造案例 3**：添加一个"翻译工具"，让 Agent 能够翻译文本
2. **扩展案例 4**：让 ReAct Agent 支持多次搜索和信息综合
3. **优化案例 5**：添加一个"质量检查员"Agent，审核最终报告
4. **组合案例**：将 RAG（检索增强）和 ReAct 结合，构建一个"智能客服"

### 推荐资源

- **官方文档**：https://langchain-ai.github.io/langgraph/
- **官方案例**：https://github.com/langchain-ai/langgraph/tree/main/examples
- **LangChain Academy**：https://academy.langchain.com/courses/intro-to-langgraph
- **Discord 社区**：https://discord.gg/langchain

---

**记住**：构建 Agent 就像搭积木，本章的 5 个案例是最基础的"积木块"。熟练掌握它们后，你就可以组合出无限可能的 AI 应用。

**下一步**：进入第 1 章，深入理解 LangGraph 的核心概念和设计哲学。